In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

In [4]:
def strategy_10_ema_crossing(data):
    data['10_EMA'] = calculate_ema(data['Adj Close'], 10)
    data['Signal'] = 0
    data['Signal'][10:] = np.where(data['Adj Close'][10:] > data['10_EMA'][10:], 1, 0)
    data['Position'] = data['Signal'].diff()
    data['Position'].replace(-1, 0, inplace=True)
    return data[['Date','Position','Stock Name']]

In [5]:
def strategy_20_ema_crossing(data):
    data['20_EMA'] = calculate_ema(data['Adj Close'], 20)
    data['Signal'] = 0
    data['Signal'][20:] = np.where(data['Adj Close'][20:] > data['20_EMA'][20:], 1, 0)
    data['Position'] = data['Signal'].diff()
    data['Position'].replace(-1, 0, inplace=True)
    return data[['Date','Position','Stock Name']]

In [6]:
def strategy_20_ema_crossing_50_ema(data):
    data['20_EMA'] = calculate_ema(data['Adj Close'], 20)
    data['50_EMA'] = calculate_ema(data['Adj Close'], 50)
    data['Signal'] = 0
    data['Signal'][50:] = np.where(data['20_EMA'][50:] > data['50_EMA'][50:], 1, 0)
    data['Position'] = data['Signal'].diff()
    data['Position'].replace(-1, 0, inplace=True)
    return data[['Date','Position','Stock Name']]

In [20]:
def strategy_10_ema_crossing_20_ema(data):
    data['10_EMA'] = calculate_ema(data['Adj Close'], 10)
    data['20_EMA'] = calculate_ema(data['Adj Close'], 20)
    data['Signal'] = 0
    data['Signal'][20:] = np.where(data['10_EMA'][20:] > data['20_EMA'][20:], 1, 0)
    data['Position'] = data['Signal'].diff()
    data['Position'].replace(-1, 0, inplace=True)
    return data[['Date','Position','Stock Name']]

In [1]:
def strategy_rsi_crossing_above_60(data, window=14, signal_value=60):
    
    delta = data['Adj Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).ewm(alpha=1/window, adjust=False).mean()
    avg_loss = pd.Series(loss).ewm(alpha=1/window, adjust=False).mean()

    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    # Generate signals: 1 when RSI crosses above 60, 0 otherwise
    data['Signal'] = np.where((data['RSI'].shift(1) < signal_value) & (data['RSI'] >= signal_value), 1, 0)

    data['Position'] = data['Signal'].diff().fillna(0)

    data['Position'].replace(-1, 0, inplace=True)

    # Return relevant columns
    return data[['Date','Position','Stock Name']]